# pytorch深度学习实践

https://www.bilibili.com/video/BV1hE411t7RN?spm_id_from=333.1007.top_right_bar_window_history.content.click&vd_source=6d033c01bacc1b94de92d9ff542bdb52

加载数据<br>如何读取数据：

In [ ]:
from PIL import Image
image_path=r"D:\python\learn_pytorch\hymenoptera_data\train\ants\0013035.jpg"
image=Image.open(image_path)
image.show()

（利用os集抓取数据集名称集(img_path是个列表，dirpath是所有图片所在的文件夹路径）

如何将每个图片的label作为txt文件写入另外一个文件夹（txt文件名与图片文件名相同）

In [ ]:
import os
root_dir=r"D:\python\learn_pytorch\hymenoptera_data\train"
target_dir="ants_img"
img_path_list=os.listdir(os.path.join(root_dir,target_dir))
label=target_dir.split('_')[0]
out_dir="ants_label"
for i in img_path_list:
    file_name=i.split('.jpg')[0]
    f=open((os.path.join(root_dir,out_dir,format(file_name))+".txt"),'w')
    f.write(label)
    f.close()

注意：open函数在文件夹中没有该文件时会自动创建一个。

tensorboard的summary writer

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer=SummaryWriter("logs")#创建实例文件夹“logs”
#writer.add_image()
for i in range(1,100):
    writer.add_scalar("y=2x",2*i,i)
 
writer.close()

如何打开tensorboard的事件文件（logs）

In [ ]:
(pytorch) D:\python\learn_pytorch>tensorboard --logdir=logs --port=6007

处理图片：

In [ ]:
from PIL import Image
img=Image.open(path)
def add_image(self, tag, img_tensor, global_step=None, walltime=None, dataformats='CHW'):
#img_tensor (torch.Tensor, numpy.array, or string/blobname): Image data

注意，add_img只能读tensor型或numpy.array型的图片，.img要先转换(先看numpy.array格式）<br>
用PIL中的Image 打开的img type是“<class 'PIL.JpegImagePlugin.JpegImageFile'>”<br>
利用numpy将img改为numpy形式:

In [ ]:
import numpy as np
img_array=np.array(img)
print(type(img_array))
#<class 'numpy.ndarray'>

注意add_image的dataformats格式，默认是CHW

In [ ]:
def add_image(self, tag, img_tensor, global_step=None, walltime=None, dataformats='CHW')

通过print(img_array.shape)以查看img是否为C（通道）H（高度）W（宽度）的形式

In [ ]:
print(img_array.shape)
#(512, 768, 3)

（由此可见img的format是HWC（通道在后），因此要把add_img语句改成

In [ ]:
writer.add_image("test",img_array,1,dataformats='HWC')

最终代码：

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from PIL import Image
writer=SummaryWriter("logs")#创建实例文件夹“logs”
img_path=r"hymenoptera_data/train/ants_img/5650366_e22b7e1065.jpg"#图片路径
img_PIL=Image.open(img_path)#打开PIL格式的图片
img_np=np.array(img_PIL)#将该图片转换为numpy.array格式
writer.add_image("test",img_np,3,dataformats='HWC')
writer.close()

torchvision中的transforms<br>
pytorch中打开structure（alt+7）可以快捷地浏览transform源文件

In [ ]:
class ToTensor:
    """Convert a ``PIL Image`` or ``numpy.ndarray`` to tensor. This transform does not support torchscript.

↑ToTensor用处:将PIL（PIL.Image）或numpy.ndarray(cv2)图片转换为tensor格式，可以用于tensorboard中的add_image(见lec7-8）

另：使用opencv：pip install opencv-python

In [ ]:
import cv2
cv_img=cv2.imread(img_path)#cv_img为numpy.ndarray格式

最终代码：

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
from torchvision import transforms
#transforms是个工具箱
writer1=SummaryWriter("log1")#实例化
path=r"hymenoptera_data/train/ants_img/Ant_1.jpg"#图片路径
img=Image.open(path)#以PIL格式打开
tensor_trans=transforms.ToTensor()#创建具体的工具"tensor_trans"
#用法：result=tool（input）
img_tensor=tensor_trans(img)
writer1.add_image("test1",img_tensor,1)
writer1.close()

常见的transform

小记def __call__与普通def的区别

In [ ]:
class person:
    def __call__(self, other):
        print(other)
    def hello(self,name):
        print("Hello"+name)
person1=person()#实例化
person1.hello("li4")
person1("zhang3")#将zhang3赋给__call__
#output：
#Helloli4
#zhang3

Normalize：

In [ ]:
img_tensor.size()
# torch.Size([3, 272, 199])

这说明img_tensor是一个有三个通道、高272、宽199的图片(rgb通道原本是[0,255]，但tensor将其压缩到了[0,1])<br>
tensor格式图片的内容：[C[H[W]]]

Normalize函数需求均值、标准差，且图片为rgb格式需求三通道。因此由以下格式：

In [ ]:
#Normalize
trans_norm=transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
print(img_tensor[0][0][0])
img_norm=trans_norm(img_tensor)
print(img_norm[0][0][0])
writer.add_image("test1",img_norm,1)
writer.close()
 
#output:
#tensor(0.3725)
#tensor(-0.2549)

Resize

改变图片大小（H，W）

In [ ]:
#Resize
trans_resize=transforms.Resize((512,512))
img_resize=trans_resize(img_tensor)
print(type(img_resize))
writer.add_image("test1",img_resize,2)

Compose(合并）

注意，由于norm与resize需要tensor数据类型，所以trans_tensor需要放在最前面

In [ ]:
#compose的变量是transforms工具的列表,实现多个功能顺序执行
trans_compose=transforms.Compose([trans_totensor,trans_resize,trans_norm])
img_compose=trans_compose(img)
writer.add_image("test1",img_compose,3)
writer.close()

RandomCrop随机裁剪

In [ ]:
#RandomCrop
trans_rc=transforms.RandomCrop((122,100))#裁剪大小
img_rc=trans_rc(img_tensor)
img_rc=trans_rc(img_tensor)
writer.add_image("test1",img_rc,4)
writer.close()

transform最终代码：

In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from torchvision import transforms

class MyData(Dataset):

    def __init__(self, root_dir, image_dir, label_dir, transform=None):
        self.root_dir = root_dir
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.label_path = os.path.join(self.root_dir, self.label_dir)
        self.image_path = os.path.join(self.root_dir, self.image_dir)
        self.image_list = os.listdir(self.image_path)
        self.label_list = os.listdir(self.label_path)
        self.transform = transform
        # 因为label 和 Image文件名相同，进行一样的排序，可以保证取出的数据和label是一一对应的
        self.image_list.sort()
        self.label_list.sort()

    def __getitem__(self, idx):
        img_name = self.image_list[idx]
        label_name = self.label_list[idx]
        img_item_path = os.path.join(self.root_dir, self.image_dir, img_name)
        label_item_path = os.path.join(self.root_dir, self.label_dir, label_name)
        img = Image.open(img_item_path)
        with open(label_item_path, 'r') as f:
            label = f.readline()

        if self.transform:
            img = transform(img)


        return img, label

    def __len__(self):
        assert len(self.image_list) == len(self.label_list)
        return len(self.image_list)


transform = transforms.Compose([transforms.Resize(400), transforms.ToTensor()])
root_dir = "dataset/train"
image_ants = "ants_image"
label_ants = "ants_label"
ants_dataset = MyData(root_dir, image_ants, label_ants, transform=transform)
image_bees = "bees_image"
label_bees = "bees_label"
bees_dataset = MyData(root_dir, image_bees, label_bees, transform=transform)

torchvision数据集

pytorch官网-docs-torchvision-packa reference下载数据集（以CIFAR为例）

torchvision.datasets.CIFAR10(root: str, train: bool = True, transform: Optional[Callable] = None, target_transform: Optional[Callable] = None, download: bool = False)

root (string) – Root directory of dataset where directory cifar-10-batches-py exists or will be saved to if download is set to True.

train (bool, optional) – If True, creates dataset from training set, otherwise creates from test set.

transform (callable, optional) – A function/transform that takes in an PIL image and returns a transformed version. E.g, transforms.RandomCrop

target_transform (callable, optional) – A function/transform that takes in the target and transforms it.

download (bool, optional) – If true, downloads the dataset from the internet and puts it in root directory. If dataset is already downloaded, it is not downloaded again.

In [ ]:
import torchvision
train_set=torchvision.datasets.CIFAR10(root="./dataset",train=True,download=True)
test_set=torchvision.datasets.CIFAR10(root="./dataset",train=False,download=True)
print(test_set[0])
img,labl=test_set[0]
print(test_set.classes)
print(img)
print(labl)
 
output:
#(<PIL.Image.Image image mode=RGB size=32x32 at 0x2C0C7D92648>, 3)
#['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
#<PIL.Image.Image image mode=RGB size=32x32 at 0x2C0C7D92588>
#3
#test的第一张图片是PIL格式，RGB形式，大小32x32，其分类为第三类

将10个trainset数据写入tensorboard：

In [ ]:
writer=SummaryWriter("p14")
for i in range (10):
    img,l=test_set[i]
    writer.add_image("set1",img,i)
writer.close()

MyDataset

In [ ]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import os
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid

writer = SummaryWriter("logs")

class MyData(Dataset):

    def __init__(self, root_dir, image_dir, label_dir, transform):
        self.root_dir = root_dir
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.label_path = os.path.join(self.root_dir, self.label_dir)
        self.image_path = os.path.join(self.root_dir, self.image_dir)
        self.image_list = os.listdir(self.image_path)
        self.label_list = os.listdir(self.label_path)
        self.transform = transform
        # 因为label 和 Image文件名相同，进行一样的排序，可以保证取出的数据和label是一一对应的
        self.image_list.sort()
        self.label_list.sort()

    def __getitem__(self, idx):
        img_name = self.image_list[idx]
        label_name = self.label_list[idx]
        img_item_path = os.path.join(self.root_dir, self.image_dir, img_name)
        label_item_path = os.path.join(self.root_dir, self.label_dir, label_name)
        img = Image.open(img_item_path)

        with open(label_item_path, 'r') as f:
            label = f.readline()

        # img = np.array(img)
        img = self.transform(img)
        sample = {'img': img, 'label': label}
        return sample

    def __len__(self):
        assert len(self.image_list) == len(self.label_list)
        return len(self.image_list)

if __name__ == '__main__':
    transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])
    root_dir = "dataset/train"
    image_ants = "ants_image"
    label_ants = "ants_label"
    ants_dataset = MyData(root_dir, image_ants, label_ants, transform)
    image_bees = "bees_image"
    label_bees = "bees_label"
    bees_dataset = MyData(root_dir, image_bees, label_bees, transform)
    train_dataset = ants_dataset + bees_dataset

    # transforms = transforms.Compose([transforms.Resize(256, 256)])
    dataloader = DataLoader(train_dataset, batch_size=1, num_workers=2)

    writer.add_image('error', train_dataset[119]['img'])
    writer.close()
    # for i, j in enumerate(dataloader):
    #     # imgs, labels = j
    #     print(type(j))
    #     print(i, j['img'].shape)
    #     # writer.add_image("train_data_b2", make_grid(j['img']), i)
    #
    # writer.close()

DataLoader

点进dataset源文件查找getitem（）可以查看该数据集返回数据类型（如CIFAR，返回img和target）

torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, sampler=None, batch_sampler=None, num_workers=0, collate_fn=None, pin_memory=False, drop_last=False, timeout=0, worker_init_fn=None, multiprocessing_context=None, generator=None, *, prefetch_factor=2, persistent_workers=False)

dataset (Dataset) – dataset from which to load the data.

batch_size (int, optional) – how many samples per batch to load (default: 1).一次取几个

shuffle (bool, optional) – set to True to have the data reshuffled at every epoch (default: False).是否打乱顺序

drop_last (bool, optional) – set to True to drop the last incomplete batch, if the dataset size is not divisible by the batch size. If False and the size of dataset is not divisible by the batch size, then the last batch will be smaller. (default: False)是否丢弃剩余

In [ ]:
import torchvision
from torch.utils.data import DataLoader
train_set=torchvision.datasets.CIFAR10(root="./dataset",transform=torchvision.transforms.ToTensor(),train=True,download=True)
train_loader=DataLoader(dataset=train_set,batch_size=4,shuffle=True,num_workers=0,drop_last=False)
for data in train_loader:
    imgs,targets=data
    print(imgs.shape)
    print(targets)
 
#output：
#torch.Size([4, 3, 32, 32])#四张图片，每张三通道，32x32
#tensor([4, 9, 3, 9]).....#标签

多张图片写入tensorboard：

In [ ]:
step=0
writer=SummaryWriter("dataloader")
for data in train_loader:
    imgs,targets=data
    writer.add_images("test1",imgs,step)
    step=step+1
writer.close()

神经网络的基本骨架-nn.Module

In [ ]:
from torch import nn
import torch
class haha(nn.Module):
    def __init__(self):
        super(haha,self).__init__()#调用父类初始化函数
        #自定义
 
    def forward(self,input):#forward一定不能拼错！否则会报错
        output=input+1#自定义
        return output#自定义
 
c=haha()#实例化
x=torch.tensor(1.0)
output=c(x)
print(output)
 
output
#tensor(2.)

torch.tensor/torch.Tensor

Tensor是class，tensor是func，将数据类型转换为Tensor

conv2d卷积

卷积核与input矩阵的各个元素乘积和生成一个output矩阵

torch.nn.functional.conv2d(input, weight, bias=None, stride=1, padding=0, dilation=1, groups=1) → Tensor

input – input tensor of shape (\text{minibatch} , \text{in\_channels} , iH , iW)(minibatch,in_channels,iH,iW) #输入内容

weight – filters of shape (\text{out\_channels} , \frac{\text{in\_channels}}{\text{groups}} , kH , kW)(out_channels,groupsin_channels​,kH,kW) #卷积核

bias – optional bias tensor of shape (\text{out\_channels})(out_channels). Default: None

stride – the stride of the convolving kernel. Can be a single number or a tuple (sH, sW). Default: 1 #每次移动几步（tuple可以规定不同方向步数不同）

padding –

implicit paddings on both sides of the input. Can be a string {‘valid’, ‘same’}, single number or a tuple (padH, padW). Default: 0 padding='valid' is the same as no padding. padding='same' pads the input so the output has the shape as the input. However, this mode doesn’t support any stride values other than 1.#填充输入

In [ ]:
import torch
import torch.nn.functional as F
#输入：5x5矩阵
input=torch.tensor([[1,2,0,3,1],
                    [0,1,2,3,1],
                    [1,2,1,0,0],
                    [5,2,3,1,1],
                    [2,1,0,1,1]])
#卷积核3x3矩阵
kernel=torch.tensor([[1,2,1],
                     [0,1,0],
                     [2,1,0]])
#尺寸变换，要对应conv2d input的要求（minibatch=1，channel=1，h，w）
input=torch.reshape(input,(1,1,5,5))
kernel=torch.reshape(kernel,(1,1,3,3))
#stride:每次（在哪个方向）移动几步
output=F.conv2d(input,kernel,stride=1)
print(output)
output1=F.conv2d(input,kernel,stride=2)
print(output1)
output2=F.conv2d(input,kernel,stride=1,padding=1)
print(output2)
 
#output
# tensor([[[[10, 12, 12],
#           [18, 16, 16],
#           [13,  9,  3]]]])
# tensor([[[[10, 12],
#           [13,  3]]]])
# tensor([[[[ 1,  3,  4, 10,  8],
#           [ 5, 10, 12, 12,  6],
#           [ 7, 18, 16, 16,  8],
#           [11, 13,  9,  3,  4],
#           [14, 13,  9,  7,  4]]]])

神经网络-卷积层Conv2d

conv_arithmetic/README.md at master · vdumoulin/conv_arithmetic · GitHub

卷积层动图理解（蓝色input，绿色output，深色kernel）

torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)

In [ ]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
 
dataset=torchvision.datasets.CIFAR10("./dataset",train=False,transform=torchvision.transforms.ToTensor(),download=True)
dataloader=DataLoader(dataset=dataset,batch_size=64,shuffle=True,drop_last=False)
class test(nn.Module):
    def __init__(self):
        super(test, self).__init__()
        self.conv1=nn.Conv2d(3,6,3,stride=1,padding=0)
 
    def forward(self,x):
        x=self.conv1(x)
        return x
writer=SummaryWriter("log1")
test1=test()
step=0
for data in dataloader:
    imgs,t=data
    output=test1(imgs)
    #torch.size([64,3,32,32])
    writer.add_images("input",imgs,step)
    #由于图片只能以三个通道显示，因此要把6个channel改成3个
    #torch.size([64,6,30,30])->[???,3,30,30]
    #batch_size不知道写多少的时候就写-1,它会自动计算
    output=torch.reshape(output,(-1,3,30,30))
    writer.add_images("output",output,step)
    step=step+1
writer.close()

Pooling layer MaxPool2

与卷积不用，池化是选取核中最大数作为output

torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)

kernel_size – the size of the window to take a max over

stride – the stride of the window. Default value is kernel_size

padding – implicit zero padding to be added on both sides

dilation – a parameter that controls the stride of elements in the window

return_indices – if True, will return the max indices along with the outputs. Useful for torch.nn.MaxUnpool2d later

ceil_mode – when True, will use ceil instead of floor to compute the output shape（True则保留多出来的input，False丢弃。）

池化：在保留数据特征的情况下减小数据量

import torch
 
import torchvision
from torch import nn
from torch.nn import MaxPool2d
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
 
dataset=torchvision.datasets.CIFAR10("./dataset",train=False,transform=torchvision.transforms.ToTensor(),download=True)
dataloader=DataLoader(dataset=dataset,batch_size=64,shuffle=True,drop_last=False)
#dtype:将数据类型更改为浮点数
#输入：5x5矩阵
input=torch.tensor([[1,2,0,3,1],
                    [0,1,2,3,1],
                    [1,2,1,0,0],
                    [5,2,3,1,1],
                    [2,1,0,1,1]],dtype=torch.float32)
input=torch.reshape(input,(-1,1,5,5))
class test(nn.Module):
    def __init__(self):
        super(test, self).__init__()
        self.maxpool1=MaxPool2d(kernel_size=3,ceil_mode=True)
    def forward(self,input):
        output=self.maxpool1(input)
        return output
test1=test()
#矩阵输出
print(test1(input))
#图像输出
writer=SummaryWriter("log2")
step=0
for data in dataloader:
    imgs,t=data
    writer.add_images("input", imgs, step)
    output=test1(imgs)
    #注意，最大池化不会改变channel（input是3通道，output也是三通道）
    writer.add_images("output", output, step)
    step=step+1
writer.close()
 
output:
# tensor([[[[2., 3.],
#           [5., 1.]]]])

非线性激活-Non-Linear Activations

ReLU:ReLU

将小于0的数据变换成0

In [ ]:
input=torch.tensor([[1,-0.5],
                    [-1,3]])
output=torch.reshape(input,(-1,1,2,2))
print(ReLU(output))
 
output:
#([[1,0],
#  [0,3]])

import torchvision
from torch import nn
from torch.nn import ReLU, Sigmoid
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
 
dataset=torchvision.datasets.CIFAR10("./dataset",train=False,transform=torchvision.transforms.ToTensor(),download=True)
dataloader=DataLoader(dataset=dataset,batch_size=64,shuffle=True,drop_last=False)
 
class test(nn.Module):
    def __init__(self):
        super(test, self).__init__()
        #inplace:是否将input替换成output
        self.relu1=ReLU(inplace=False)
        self.sigmoid1=Sigmoid()
    def forward(self,input):
        output=self.sigmoid1(input)
        output=self.sigmoid1(output)
        return output
test1=test()
 
writer=SummaryWriter("log2")
step=0
for data in dataloader:
    imgs,t=data
    writer.add_images("input", imgs, step)
    output=test1(imgs)
    writer.add_images("output", output, step)
    step=step+1
writer.close()

线性层

torch.nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)

in_features – size of each input sample

out_features – size of each output sample

bias – If set to False, the layer will not learn an additive bias. Default: True

将图片内容转化为一维：

In [ ]:
for data in dataloader:
    imgs,t=data
    print(imgs.shape)
    output=torch.reshape(imgs,(64,1,1,-1))
    print(output.shape)
 
output：
#torch.Size([64, 3, 32, 32])
#torch.Size([64, 1, 1, 3072])

由此可见，将图片展开后，也就是inputsize为3072

In [ ]:
import torch
import torchvision
from torch import nn
from torch.nn import Linear
from torch.utils.data import DataLoader
 
dataset=torchvision.datasets.CIFAR10("dataset",train=False,transform=torchvision.transforms.ToTensor(),
                                    download=True)
#这里不droplast后面会报错，因为linear1的定义是196608，而最后一组图片不够64张，最后的大小也不足196608
dataloader=DataLoader(dataset,batch_size=64,drop_last=True)
 
class test(nn.Module):
    def __init__(self):
        super(test, self).__init__()
        self.linear1=Linear(3072,10)
    def forward(self,input):
        output=self.linear1(input)
        return output
test1=test()
for data in dataloader:
    imgs,t=data
    print(imgs.shape)
    #将图片线性化
    output=torch.reshape(imgs,(64,1,1,-1))
    #flatten也可以用
    # output=torch.flatten(imgs)
    print(output.shape)
    output=test1(output)
    print(output.shape)
 
output：
#torch.Size([64, 3, 32, 32])
#torch.Size([64, 1, 1, 3072])
#torch.Size([64, 1, 1, 10])

CIFAR10模型结构

In [ ]:
import torch
from torch import nn
 
class test(nn.Module):
    def __init__(self):
        super(test, self).__init__()
        #因为size_in和size_out都是32，经过计算得出padding=2，stride=1
        self.conv1=nn.Conv2d(3,32,5,padding=2,stride=1)
        self.pool1=nn.MaxPool2d(2)
        #尺寸不变，和上面一样
        self.conv2=nn.Conv2d(32,32,5,stride=1,padding=2)
        self.pool2=nn.MaxPool2d(2)
        # 尺寸不变，和上面一样
        self.conv3=nn.Conv2d(32,64,5,stride=1,padding=2)
        self.pool3 = nn.MaxPool2d(2)
        self.flatten=nn.Flatten()
        #in_feature:64*4*4,out_feature:64
        self.linear1=nn.Linear(1024,64)
        self.linear2=nn.Linear(64,10)
    def forward(self,x):
        x=self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.pool3(x)
        x=self.flatten(x)
        x=self.linear1(x)
        x=self.linear2(x)
        return x
test1=test()
#对网络结构进行检验
input=torch.ones((64,3,32,32))
output=test1(input)
print(output.shape)
 
output：
#torch.Size([64, 10])

可以用nn.Sequential将代码变得更简洁：

In [ ]:
class test(nn.Module):
    def __init__(self):
        super(test, self).__init__()
 
        self.model1=nn.Sequential(
            nn.Conv2d(3, 32, 5, padding=2, stride=1),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, stride=1, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, stride=1, padding=2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(1024, 64),
            nn.Linear(64, 10)
        )
 
    def forward(self,x):
        x=self.model1(x)
        return x

另：用SummaryWriter 的 add graph也可以可视化*比print高级一些

损失函数与反向传播

torch.nn-loss functions

注意输入和输出的形式

torch.nn.CrossEntropyLoss(weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='mean')

input has to be a Tensor of size either (minibatch, C)(minibatch,C) or (minibatch, C, d_1, d_2, ..., d_K)(minibatch,C,d1​,d2​,...,dK​)

输入为神经网络最后对该图片的每种类型预测的概率（minibatch），与该图片的类型标签

In [ ]:
test1=test()
loss=nn.CrossEntropyLoss()
for data in dataloader:
    imgs,t=data
    #output代表该图像是每种类别的概率
    output=test1(imgs)
    print(output)
    print(t)
    #实际输出和目标之间的差距
    result_loss=loss(output,t)
 
output：
# tensor([[-0.0928, -0.0330,  0.0529,  0.1399, -0.1088,  0.0506, -0.0798,  0.0198,
#           0.1653, -0.0215]], grad_fn=<AddmmBackward>)
# tensor([7])
# tensor(2.2961, grad_fn=<NllLossBackward>)

优化器

pytorch-optim

在degug内查看gard：

test1-conv1（或其他网络）-weight-grad

In [ ]:
test1=test()
#lossFunction模型
loss=nn.CrossEntropyLoss()
#优化器模型
optim=torch.optim.SGD(test1.parameters(),0.01)
#只对每张图片进行一轮学习
for data in dataloader:
    imgs,t=data
    output=test1(imgs)
    result_loss=loss(output,t)
    #将每个梯度清为0（初始化）
    optim.zero_grad()
    #反向传播，得到每个可调节参数对应的梯度（grad不再是none）
    result_loss.backward()
    #对每个参数进行改变，weight-data被改变
    optim.step()
    print(result_loss)

尝试进行多次优化：

In [ ]:
test1=test()
#lossFunction模型
loss=nn.CrossEntropyLoss()
#优化器模型
optim=torch.optim.SGD(test1.parameters(),0.01)
#进行20次优化
for epcho in range (20):
    running_loss=0.0
    for data in dataloader:
        imgs,t=data
        output=test1(imgs)
        result_loss=loss(output,t)
        #将每个梯度清为0（初始化）
        optim.zero_grad()
        #反向传播，得到每个可调节参数对应的梯度（grad不再是none）
        result_loss.backward()
        #对每个参数进行改变，weight-data被改变
        optim.step()
        #计算每轮优化中每个变量的loss和
        running_loss=running_loss+result_loss
    print(running_loss)
 
output：
#总loss在逐渐变小
# tensor(18712.0938, grad_fn= < AddBackward0 >)
# tensor(16126.7949, grad_fn= < AddBackward0 >)
# tensor(15382.0703, grad_fn= < AddBackward0 >)

现有网络模型使用及修改

vgg16：

In [ ]:
vgg16_f=torchvision.models.vgg16(pretrained=False)
vgg16_t=torchvision.models.vgg16(pretrained=True)

可以看到vgg16最后输出1000个类，想要将这1000个类改成10个类：

In [ ]:
from torch import nn
 
vgg16_f=torchvision.models.vgg16(pretrained=False)
vgg16_t=torchvision.models.vgg16(pretrained=True)
#改动网络
#添加
vgg16_t.add_module("add_linear",nn.Linear(1000,10))
#或改动
vgg16_f.classifier[6]=nn.Linear(4096,10)
print(vgg16_t)
print(vgg16_f)

网络模型的保存与读取

保存：（以vgg16为例）

In [ ]:
import torch
import torchvision
vgg16=torchvision.models.vgg16(pretrained=False)
#方式1（.pth不是必须的）
torch.save(vgg16,"vgg16.pth")
#方式2：将参数保存为字典格式（官方推荐）
torch.save(vgg16.state_dict(),"vgg16_2.pth")

读取：

In [ ]:
import torch
import torchvision
vgg16=torchvision.models.vgg16(pretrained=False)
#方式一=》保存方式一(路径)
model=torch.load("vgg16.pth")
print(model)
#方式2=》字典格式
#注意方式2要printvgg16这个模型，不能model=xxx，print（model）
vgg16.load_state_dict(torch.load("vgg16_2.pth"))
print(vgg16)

注意，以方式一形式读取自定义模型时要先将该模型复制或引用到读取文件中，否则会报错

完整模型训练

准备数据，加载数据，搭建网络，创建网络，设置损失函数，设置优化器，设置训练参数（epoch，训练次数，测试次数）开始训练（取数据，进网络，算loss，梯度清零，反向传播，网络优化），最后验证（无梯度，取数据，进网络，算loss，loss求和，求精度，精度求和），结果聚合展示



载入模型、数据

加载数据

创建网络模型

初始化损失函数、优化器

设置训练网络的参数（训练、测试次数、轮数）

添加tensorboard（可视化）

for epoch：

        for data in trainloader：

                分割数据，loss，清零，反向，优化

        for data in testloader：

                分割数据，计算准确度

展现每轮总loss，特定步数loss，保存模型

准确度计算思路：找出最大预测结果与target相同的图片数量，用其除以图片总数


测试思路：path，open，convert，transform（包含resize和totensor），网络模型，加载网络模型，reshape，无梯度，argmax


In [ ]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
#import自己写的models
from p27_1model import *
 
train_data=torchvision.datasets.CIFAR10("../datasets",train=True,transform=torchvision.transforms.ToTensor(),
                                        download=True)
test_data=torchvision.datasets.CIFAR10("../datasets",train=False,transform=torchvision.transforms.ToTensor(),
                                        download=True)
#查看数据集长度
train_size=len(train_data)
test_size=len(test_data)
print("size of train,test is:{},{}".format(train_size,test_size))
#利用dataloader加载
train_dataloader=DataLoader(train_data,64)
test_dataloader=DataLoader(test_data,64)
#创建网络模型
test1=test()
#损失函数
loss_f=nn.CrossEntropyLoss()
#优化器
#1e-2=0.01
learning_rate=1e-2
opt=torch.optim.SGD(test1.parameters(),lr=learning_rate,)
#设置训练网络的参数
#记录训练次数
train_step=0
#测试次数
test_step=0
#训练轮数
epoch=10
#添加tensoeboard
writer=SummaryWriter("train_log")
 
 
for i in range(epoch):
    print("第{}轮训练开始".format(i+1))
    #训练步骤开始
    #有时不必要：test1.train()
    for data in train_dataloader:
        imgs,t=data
        output=test1(imgs)
        loss=loss_f(output,t)
        #优化器优化模型
        opt.zero_grad()
        loss.backward()
        opt.step()
        train_step=train_step+1
        #loss.item更加规范（.item不会打印数据类型，例如tensor(5))
        if train_step%100==0:
            print("训练次数{}，loss值为{}".format(train_step,loss.item()))
            writer.add_scalar("train_loss",loss.item(),train_step)
    loss_total=0
    #测试步骤开始
    #有时不必要：test1.eval()
    total_correct=0
    with torch.no_grad():
        for data in test_dataloader:
            imgs,t=data
            output=test1(imgs)
            loss=loss_f(output,t)
            loss_total=loss_total+loss.item()
            test_step=test_step+1
            #argmax参数：1为横向比较，2为纵向比较，output为64,10的矩阵
            #output.argmax(1)==t是为了得到[Ture,False,True....]这种形式
            #.sum:T为1，F为0
            corect=(output.argmax(1)==t).sum()
            total_correct=total_correct+corect
            accuracy=total_correct/test_size
    print("测试集总loss{}".format(loss_total))
    writer.add_scalar("test_loss",loss_total,test_step)
    writer.add_scalar("accuracy",accuracy,test_step)
 
    torch.save(test1,"test1{}.pth".format(i))
    print("模型已保存")
 

利用gpu训练

改动：

网络模型创建实例、损失函数、分割数据集（输入和标注）时加.cuda()

In [ ]:
test1=test()
if torch.cuda.is_available()==True:
    test1.cuda()

In [ ]:
loss_f=nn.CrossEntropyLoss()
if torch.cuda.is_available()==True:
    loss_f=loss_f.cuda()

In [ ]:
    for data in train_dataloader:
        imgs,t=data
        if torch.cuda.is_available() == True:
            imgs=imgs.cuda()
            t=t.cuda()

MNIST数据集训练：

In [ ]:
import os
import shutil

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

# 定义训练设备
device = torch.device("cuda")
# 准备训练集、测试集
train_dataset = datasets.MNIST(root="MNIST", train=True, download=True,
                               transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root="MNIST", train=False, download=True,
                              transform=transforms.ToTensor())
# len
train_data_size = len(train_dataset)
test_data_size = len(test_dataset)
# 用dataloader加载数据
train_dataloader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=True)


# 搭建网络模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Sequential(torch.nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(),
                                         torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(),
                                         torch.nn.MaxPool2d(stride=2, kernel_size=2))
        self.dense = torch.nn.Sequential(torch.nn.Linear(14 * 14 * 128, 1024),
                                         torch.nn.ReLU(),
                                         torch.nn.Dropout(p=0.5),
                                         torch.nn.Linear(1024, 10))

    def forward(self, x):
        x = self.conv1(x)
        x = x.view(-1, 14 * 14 * 128)
        x = self.dense(x)
        return x


# 创建网络模型
net = Net()
net = net.to(device)
# 定义损失函数
loss_func = nn.CrossEntropyLoss()
loss_func = loss_func.to(device)
# 定义优化器
learning_rate = 1e-3
optimizer = torch.optim.Adam(params=net.parameters(), lr=learning_rate)
# 设置训练网络的一些参数
# 记录训练次数
total_train_step = 0
# 记录测试次数
total_test_step = 0
# 训练轮数
epoch = 5
# 添加tensorboard
logs_path = "logs_mnist"
if os.path.exists(logs_path):
    shutil.rmtree(logs_path)
writer = SummaryWriter(logs_path)

for i in range(epoch):
    print("------第{}轮训练开始------".format(i + 1))
    net.train()
    for data in train_dataloader:
        images, targets = data
        images = images.to(device)
        targets = targets.to(device)
        opt = net(images)
        loss = loss_func(opt, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # 一轮训练结束
        total_train_step += 1
        if total_train_step % 100 == 0:
            print("训练次数:{},loss:{}".format(total_train_step, loss))
            writer.add_scalar("train_loss", loss.item(), total_train_step)
    net.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            images, targets = data
            images = images.to(device)
            targets = targets.to(device)
            opt = net(images)
            loss = loss_func(opt, targets)
            total_test_loss += loss
            accuracy = (opt.argmax(1) == targets).sum()
            total_accuracy += accuracy
        print("第{}轮训练整体测试集上的loss：{}".format(total_test_step + 1, total_test_loss))
        print("第{}轮训练整体测试集上的正确率：{}".format(total_test_step + 1, total_accuracy / test_data_size))
        writer.add_scalar("test_loss", total_test_loss, total_train_step)
        writer.add_scalar("test_accuracy", total_accuracy / test_data_size, total_test_step)
        if (total_accuracy / test_data_size) >= 0.99:
            torch.save(net, "mnist.pth")
        total_test_step += 1


bp神经网络

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

# prepare dataset
xy = np.loadtxt('diabetes.csv.gz', delimiter=',', dtype=np.float32)
x_data = torch.from_numpy(xy[:, :-1])  # 第一个‘：’是指读取所有行，第二个‘：’是指从第一列开始，最后一列不要
y_data = torch.from_numpy(xy[:, [-1]])  # [-1] 最后得到的是个矩阵
# design model using class


class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = torch.nn.Linear(8, 6)  # 输入数据x的特征是8维，x有8个特征
        self.linear2 = torch.nn.Linear(6, 4)
        self.linear3 = torch.nn.Linear(4, 1)
        self.sigmoid = torch.nn.Sigmoid()  # 将其看作是网络的一层，而不是简单的函数使用

    def forward(self, x):
        x = self.sigmoid(self.linear1(x))
        x = self.sigmoid(self.linear2(x))
        x = self.sigmoid(self.linear3(x))  # y hat
        return x


model = Model()

# construct loss and optimizer
# criterion = torch.nn.BCELoss(size_average = True)
criterion = torch.nn.BCELoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

epoch_list = []
loss_list = []
# training cycle forward, backward, update
for epoch in range(100):
    y_pred = model(x_data)
    loss = criterion(y_pred, y_data)
    accuracy = (y_pred.argmax(1) == y_data).sum()
    print("epoch:", epoch, "loss:", loss.item())
    epoch_list.append(epoch)
    loss_list.append(loss.item())

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

plt.plot(epoch_list, loss_list)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()
